In [1]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

### Data exploration

In [3]:
#Let's get all the data to understand the available fields and its types
query = '''SELECT * FROM fb55.citibike; '''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


In [4]:
table.describe()

/opt/conda/envs/PUI2016_Python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


,the_geom,the_geom_webmercator,field_1,start_station_longitude,tripduration,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_latitude,end_station_longitude,bikeid,birth_year,gender
count,0.0,0.0,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,46200.000000,45658.000000,46200.000000
mean,NaN,NaN,23099.500000,-73.990094,675.865823,439.865216,23100.500000,40.735696,440.415714,40.735391,-73.990422,18147.894848,1975.352775,1.158030
std,NaN,NaN,13336.935555,0.011493,938.842345,328.568800,13336.935555,0.018554,328.283659,0.018492,0.011505,2047.763519,11.686298,0.395904
min,NaN,NaN,0.000000,-74.017134,60.000000,72.000000,1.000000,40.680342,72.000000,40.680342,-74.017134,14530.000000,1900.000000,0.000000
25%,NaN,NaN,11549.750000,-73.997901,354.000000,306.000000,11550.750000,40.723684,307.000000,40.723180,-73.998102,16409.250000,NaN,1.000000
50%,NaN,NaN,23099.500000,-73.990093,529.000000,417.000000,23100.500000,40.737815,416.000000,40.737050,-73.990697,18125.000000,NaN,1.000000
75%,NaN,NaN,34649.250000,-73.981854,796.000000,491.000000,34650.250000,40.750200,491.000000,40.750073,-73.981923,19870.250000,NaN,1.000000
max,NaN,NaN,46199.000000,-73.950048,43016.000000,3002.000000,46200.000000,40.771522,3002.000000,40.771522,-73.950048,21699.000000,1999.000000,2.000000


### Task 1 - Familiarize with SQL Clauses

In [5]:
#Sort data by start_station_id, tripduration
#Only checking trips with duration <= 3 hours
query = '''SELECT * FROM fb55.citibike
WHERE tripduration <= 3*60*60
ORDER BY start_station_id ASC, tripduration ASC;'''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980.0,2
1,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959.0,2
2,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995.0,1
3,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970.0,1
4,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983.0,1


In [6]:
#Only show the top 10 records (aka head in SQL)
query = '''SELECT * FROM fb55.citibike
WHERE tripduration <= 3*60*60
ORDER BY start_station_id ASC, tripduration ASC
LIMIT 10;'''
table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980,2
1,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959,2
2,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995,1
3,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970,1
4,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983,1
5,NaN,W 52 St & 11 Ave,NaN,32603,-73.993929,261,2015-02-06 08:58:00+00,2015-02-06 09:02:00+00,72,32566,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,18492,Subscriber,1971,1
6,NaN,W 52 St & 11 Ave,NaN,16015,-73.993929,276,2015-02-04 12:46:00+00,2015-02-04 12:51:00+00,72,15971,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,19216,Subscriber,1961,1
7,NaN,W 52 St & 11 Ave,NaN,41366,-73.993929,283,2015-02-07 12:10:00+00,2015-02-07 12:15:00+00,72,41355,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,21588,Subscriber,1974,1
8,NaN,W 52 St & 11 Ave,NaN,42084,-73.993929,288,2015-02-07 13:50:00+00,2015-02-07 13:54:00+00,72,42074,40.767272,514,12 Ave & W 40 St,40.760875,-74.002777,19124,Subscriber,1978,1
9,NaN,W 52 St & 11 Ave,NaN,2348,-73.993929,301,2015-02-01 13:08:00+00,2015-02-01 13:13:00+00,72,2282,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,18174,Subscriber,1974,1


In [7]:
#Only show the last 10 records (aka tail in SQL)
query = '''SELECT * FROM fb55.citibike
WHERE tripduration <= 3*60*60
ORDER BY start_station_id DESC, tripduration DESC
LIMIT 10;'''
table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,South End Ave & Liberty St,NaN,1576,-74.015756,7225,2015-02-01 11:46:00+00,2015-02-01 13:47:00+00,3002,1506,40.711512,499,Broadway & W 60 St,40.769155,-73.981918,21202,Customer,NaN,0
1,NaN,South End Ave & Liberty St,NaN,8891,-74.015756,2139,2015-02-03 09:33:00+00,2015-02-03 10:09:00+00,3002,8831,40.711512,510,W 51 St & 6 Ave,40.760660,-73.980420,15552,Subscriber,1958.0,1
2,NaN,South End Ave & Liberty St,NaN,23568,-74.015756,2131,2015-02-05 08:42:00+00,2015-02-05 09:17:00+00,3002,23529,40.711512,359,E 47 St & Park Ave,40.755103,-73.974987,21473,Subscriber,1985.0,1
3,NaN,South End Ave & Liberty St,NaN,10232,-74.015756,1977,2015-02-03 16:04:00+00,2015-02-03 16:37:00+00,3002,10173,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,15309,Subscriber,1969.0,1
4,NaN,South End Ave & Liberty St,NaN,27243,-74.015756,1977,2015-02-05 16:11:00+00,2015-02-05 16:44:00+00,3002,27203,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,20069,Subscriber,1969.0,1
5,NaN,South End Ave & Liberty St,NaN,6846,-74.015756,1976,2015-02-02 09:21:00+00,2015-02-02 09:53:00+00,3002,6781,40.711512,494,W 26 St & 8 Ave,40.747348,-73.997236,19665,Subscriber,1959.0,1
6,NaN,South End Ave & Liberty St,NaN,44708,-74.015756,1920,2015-02-07 18:36:00+00,2015-02-07 19:08:00+00,3002,44703,40.711512,410,Suffolk St & Stanton St,40.720664,-73.985180,15443,Subscriber,1964.0,2
7,NaN,South End Ave & Liberty St,NaN,11648,-74.015756,1908,2015-02-03 18:54:00+00,2015-02-03 19:26:00+00,3002,11587,40.711512,279,Peck Slip & Front St,40.707873,-74.001670,19133,Subscriber,1996.0,1
8,NaN,South End Ave & Liberty St,NaN,1297,-74.015756,1900,2015-02-01 11:14:00+00,2015-02-01 11:45:00+00,3002,1228,40.711512,525,W 34 St & 11 Ave,40.755942,-74.002116,17849,Customer,NaN,0
9,NaN,South End Ave & Liberty St,NaN,17637,-74.015756,1830,2015-02-04 16:33:00+00,2015-02-04 17:04:00+00,3002,17590,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,20246,Subscriber,1969.0,1


In [8]:
#List all unique start_station_id values 
query = '''SELECT DISTINCT start_station_id FROM fb55.citibike;'''
table = get_data(query)
table.head(5)

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


In [9]:
#let's clean that table
table = table.loc[:,['start_station_id']]
table.head()

,start_station_id
0,120
1,285
2,251
3,195
4,453


In [10]:
#Aggregation functions: Count the number of trips (aka wc -l in SQL)
query = '''SELECT COUNT(*) as trip_count FROM fb55.citibike;'''
table = get_data(query)
table

,trip_count,Unnamed: 1
0,46200,NaN


In [11]:
#let's clean that table
table = table.loc[:,['trip_count']]
table

,trip_count
0,46200


In [12]:
#Aggregation functions:  Find the average/min/max trip duration
query = '''SELECT AVG(tripduration) as avg_tripduration,
MIN(tripduration) as min_tripduration,
MAX(tripduration) as max_tripduration
FROM fb55.citibike;'''
table = get_data(query)
table

,avg_tripduration,min_tripduration,max_tripduration
0,675.865823,60,43016


### Task 2 - Working with date/time

In [13]:
#Selecting trips started on Feb-02-2015 only 
query = '''SELECT * FROM fb55.citibike
WHERE starttime::date = '2015-02-02';'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [14]:
#Selecting trips started on the weekends
query = '''SELECT * FROM fb55.citibike
WHERE EXTRACT(ISODOW FROM starttime) IN (6,7);'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


In [15]:
#What are average trip duration during weekends? 
query = '''SELECT AVG(tripduration) as avg_tripduration FROM fb55.citibike
WHERE EXTRACT(ISODOW FROM starttime) IN (6,7);'''
table = get_data(query)
table.head()

,avg_tripduration,Unnamed: 1
0,662.942181,NaN


In [16]:
#let's clean that table
table = table.loc[:,['avg_tripduration']]
table

,avg_tripduration
0,662.942181


In [17]:
#Can we do the same for weekday?
query = '''SELECT * FROM fb55.citibike
WHERE EXTRACT(ISODOW FROM starttime) NOT IN (6,7);'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 39 St & 3 Ave,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,8263,40.748901,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1
3,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
4,NaN,E 2 St & 2 Ave,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,10411,40.725029,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1


In [18]:
#Average again
query = '''SELECT AVG(tripduration) as avg_tripduration FROM fb55.citibike
WHERE EXTRACT(ISODOW FROM starttime) NOT IN (6,7);'''
table = get_data(query)
table = table.loc[:,['avg_tripduration']]
table

,avg_tripduration
0,681.052292


### Task 3 -  Working with Space

In [19]:
#Showing the list of start station locations, using GROUP BY
query = '''SELECT start_station_id FROM fb55.citibike
GROUP BY start_station_id;'''
table = get_data(query)
table.head(5)

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


In [20]:
table = table.loc[:,['start_station_id']]
table.head()

,start_station_id
0,120
1,285
2,251
3,195
4,453


In [21]:
#Showing the number of trips started per station 
query = '''SELECT start_station_id, COUNT(*) as trip_count FROM fb55.citibike
GROUP BY start_station_id;'''
table = get_data(query)
table.head(5)

,start_station_id,trip_count
0,120,17
1,285,373
2,251,194
3,195,137
4,453,218


In [22]:
#But only for stations within 500m of Time Square!
#The coordinates of Time Square is (40.7577,-73.9857)
query = '''SELECT start_station_id, COUNT(*) as trip_count FROM fb55.citibike
WHERE ST_DISTANCE(ST_Point(-73.9857,40.7577)::geography,
  ST_Point(start_station_longitude,start_station_latitude)::geography) <= 500
GROUP BY start_station_id;'''
table = get_data(query)
table

,start_station_id,trip_count
0,173,213
1,493,141
2,2021,141
3,529,221
4,524,112
5,477,507
6,465,251


### Task 4 - Putting it all together

In [23]:
#Find the station that had the longest average trip duration during
#weekends and within 500m of TimeSquare.
query = '''SELECT start_station_id, AVG(tripduration) as avg_tripduration FROM fb55.citibike
WHERE ST_DISTANCE(ST_Point(-73.9857,40.7577)::geography,
  ST_Point(start_station_longitude,start_station_latitude)::geography) <= 500
  AND EXTRACT(ISODOW FROM starttime) IN (6,7)
GROUP BY start_station_id
ORDER BY avg_tripduration DESC
LIMIT 1;'''
table = get_data(query)
table

,start_station_id,avg_tripduration
0,173,1010.104167


In [24]:
#Extra: create lines for trips started from stations within 500m of Times
#Squares and lasted less than 2 hours. The number of trips per each
#pair of stations are output as attributes of these lines.
query = '''SELECT CONCAT_WS('-' ,start_station_id, end_station_id) as line_from_to, COUNT(*) as trip_count FROM fb55.citibike
WHERE ST_DISTANCE(ST_Point(-73.9857,40.7577)::geography,
  ST_Point(start_station_longitude,start_station_latitude)::geography) <= 500
  AND tripduration < 2*60*60
GROUP BY line_from_to;'''
table = get_data(query)
table.head()

,line_from_to,trip_count
0,477-212,1
1,477-168,3
2,529-352,6
3,529-515,9
4,2021-137,2
